# Turfs and Counts of High Density Points

Add 311 counts to the turfs data frame.

**Note:** This is another nb that needs to be revisited!!

In [ ]:
brooklyn_turfs_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-turfs.parq')

In [ ]:
brooklyn_turfs_subset_gdf = brooklyn_turfs_gdf[['OrgName', 'OrgWebSite', 'PrimST', 'PopID', 'Shape_Area', 'geometry']].copy().to_crs('epsg:2263')

In [ ]:
pertinent_turfs_primst = ['Waterfront / Beach / Shoreline', 
                          'Watershed / Sewershed', 
                          'Stream / River / Canal', 
                          'Salt Marsh', 
                          'Public Right of Way (Sidewalk, street ends, traffic island, public plaza)', 
                          'Freshwater Wetland']

This is wrong.  Need to compute row['geometry'].area / 27878400 to get sq_miles.  Later.

In [ ]:
primst_turfs_gdf = brooklyn_turfs_subset_gdf.query(f"PrimST in @pertinent_turfs_primst").reset_index().drop(columns='index')
primst_turfs_gdf['sq_miles'] = primst_turfs_gdf.apply(lambda row: row['geometry'].area / 27878400, axis=1)
primst_turfs_gdf.sort_values('sq_miles', inplace=True, ascending=False)

In [ ]:
more_than_three_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-311-elevation.parq')
len(more_than_three_gdf)

So there are 243 points with more than 3 311 requests for Sewer type.

In [ ]:
type(more_than_three_gdf)

In [ ]:
type(primst_turfs_gdf)

more_than_three_gdf are Points, and primst_turfs_gdf are Poly's

In [ ]:
more_than_three_gdf = more_than_three_gdf.set_crs('epsg:4326')

primst_turfs_gdf = primst_turfs_gdf.to_crs('epsg:4326')

In [ ]:
#primst_turfs_gdf

In [ ]:
primst_turfs_gdf = primst_turfs_gdf.reset_index().drop(columns='index')

primst_turfs_gdf is now sorted by sq_miles and the right crs

In [ ]:
#primst_turfs_gdf

In [ ]:
def request_count(a_turf):
    requests_in_turf = more_than_three_gdf.sjoin(a_turf, how='inner', predicate='within')
    return len(requests_in_turf)

In [ ]:
#primst_turfs_gdf['request_count'] = primst_turfs_gdf.apply(lambda row: request_count(row.iloc[[0]]), axis=1)

In [ ]:
primst_turfs_gdf['request_count'] = 0

In [ ]:
import pdb
def messy(turfs_gdf):
    results_gdf = turfs_gdf.copy()
    for i in results_gdf.index:
        a_turf_gdf = results_gdf.iloc[[i]]
        #print(i, type(a_turf_gdf))
        #pdb.set_trace()
        requests_in_turf = more_than_three_gdf.to_crs('epsg:4326').sjoin(a_turf_gdf.to_crs('epsg:4326'), how='inner', predicate='within')
        #primst_turfs_gdf.iloc[[i]]['request_count'] = len(requests_in_turf)
        #turfs_gdf.iloc[i]['request_count'] = len(requests_in_turf)
        results_gdf.at[i, 'request_count'] = len(requests_in_turf)
        #print(f"iter: {i} {len(requests_in_turf)}")
        #pdb.set_trace()
        #print(turfs_gdf['OrgName'], len(requests_in_turf))
    #pdb.set_trace()
    return results_gdf

In [ ]:
primst_with_counts_gdf = messy(primst_turfs_gdf)

In [ ]:
primst_with_counts_gdf

In [ ]:
primst_with_counts_gdf.request_count.value_counts()

In [ ]:
primst_with_counts_gdf.columns

In [ ]:
primst_with_counts_gdf.to_parquet('../data/processed/brooklyn/primst-turfs-counts.parq')